In [6]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import praw
from pytrends.request import TrendReq
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import matplotlib.dates as mdates
from scipy import interpolate
import feedparser
from datetime import datetime, timedelta

nltk.download('punkt')
nltk.download('stopwords')



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/julianramirez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/julianramirez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/julianramirez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
end_date = datetime.today()

start_date = end_date - timedelta(days=720)

start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

print("Start Date:", start_date_str)
print("End Date:", end_date_str)

ipc = yf.download('^MXX', start=start_date_str, end=end_date_str, interval='1d')
ipc.reset_index(inplace=True)
ipc['Date'] = pd.to_datetime(ipc['Date'])

Start Date: 2022-11-12
End Date: 2024-11-01


[*********************100%***********************]  1 of 1 completed


In [31]:
def get_google_news_rss_feed(feed_url):
    feed = feedparser.parse(feed_url)
    if feed.bozo:
        print("Error parsing feed:", feed.bozo_exception)
        return []
    if not feed.entries:
        print("No entries found in the feed.")
        return []
    articles = []
    for entry in feed.entries:
        title = entry.title if 'title' in entry else 'No title'
        summary = entry.summary if 'summary' in entry else 'No summary'
        published = datetime(*entry.published_parsed[:6]) if 'published_parsed' in entry else None
        articles.append({
            'title': title,
            'content': summary,
            'date': published
        })
    return articles

# Collect news articles
bolsa_rss_url = "https://news.google.com/rss/search?q=Bolsa+Mexicana+de+Valores&hl=es-419&gl=MX&ceid=MX:es-419"
news_articles = get_google_news_rss_feed(bolsa_rss_url)
if news_articles:
    print(f"Collected {len(news_articles)} articles.")
else:
    print("No articles were collected.")

def preprocess_text(text, stop_words):
    if not isinstance(text, str):
        text = ''
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-záéíóúñ\s]', '', text)
    tokens = word_tokenize(text, language='spanish')
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

#sentiment analysis model
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


def get_sentiment_score(text):
    if not text:
        return 0.0
    try:
        result = sentiment_pipeline(text[:512])[0]  # Truncate text to 512 tokens
        label = result['label']
        score = int(label.split()[0])  # Extract the number from '1 star', '2 stars', etc.
        normalized_score = (score - 3) / 2  # Normalize to range [-1, 1]
        return normalized_score
    except Exception as e:
        print(f"Error processing text: {e}")
        return 0.0

news_df = pd.DataFrame(news_articles)
if 'date' in news_df.columns:
    news_df['Date'] = pd.to_datetime(news_df['date']).dt.normalize()
    news_df.drop('date', axis=1, inplace=True)
else:
    news_df['Date'] = pd.to_datetime(news_df['published']).dt.normalize()
news_df['combined_text'] = news_df['title'].astype(str) + ' ' + news_df['content'].astype(str)
news_df['content_clean'] = news_df['combined_text'].apply(lambda x: preprocess_text(x, stop_words))
news_df['sentiment_score'] = news_df['content_clean'].apply(get_sentiment_score)

desired_columns = ['Date', 'title']

news_df[desired_columns].to_csv('news_titles.csv', index=False)

print("Date and titles have been saved to 'news_titles.csv'.")



Collected 87 articles.


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Date and titles have been saved to 'news_titles.csv'.


In [28]:

#Reddit API 
reddit = praw.Reddit(
    client_id="fbtWruG8aopQ5chxNLpURw",
    client_secret="hGgoXi6sA0Qv0wgAezpdXqGvOgc_1Q",
    user_agent="marketsentimentbmv"
)

reddit.read_only = True

def get_reddit_posts(subreddits, query, limit=10):
    posts = []
    for subreddit in subreddits:
        print(f"Searching in subreddit: {subreddit}")
        subreddit_obj = reddit.subreddit(subreddit)
        for submission in subreddit_obj.search(query, limit=limit):
            posts.append({
                'title': submission.title,
                'content': submission.selftext,
                'created': pd.to_datetime(submission.created_utc, unit='s')
            })
    return posts

subreddits = ['MexicoBursatil', 'MexicoFinanciero']
query = 'Acciones' , 'comprar', 'vender'
reddit_posts = get_reddit_posts(subreddits, query)

Version 7.7.1 of praw is outdated. Version 7.8.1 was released 7 days ago.


Searching in subreddit: MexicoBursatil
Searching in subreddit: MexicoFinanciero


In [46]:

# reddit posts to DataFrame
reddit_df = pd.DataFrame(reddit_posts)

reddit_df['Date'] = reddit_df['created'].dt.normalize()
reddit_df.drop('created', axis=1, inplace=True)

# compute sentiment
reddit_df['content_clean'] = reddit_df.apply(lambda x: preprocess_text(x['title'] + ' ' + x['content'], stop_words), axis=1)
reddit_df['sentiment_score'] = reddit_df['content_clean'].apply(get_sentiment_score)


                                                title  \
0   10 Acciones Que Warren Buffett Acaba De Compra...   
1   De los errores se aprende y cuando se aprende ...   
2   Creen que bitcoin baje mas? Es buen momento pa...   
3                                   Venta de acciones   
4   Vale la pena meterle al fondo de Gbm? (GBMMINT...   
5                    Me gané unas acciones, y ahora?    
6                                 Etfs de acumulación   
7                                   Inversión en QQQM   
8   ¿Cuál es el funcionamiento de la inversión en ...   
9   ¿Qué son los “Fully Paid Lending programs” y e...   
10  Madres... si hubiera comprado gansitos en 2018...   
11  Que tan rentable es vender carros al sur de Me...   
12               Vender bitcoin para meterlo a cetes?   
13  Como lidiar con la frustacion de vender accion...   
14  ¿Puedo vender mi casa a un familiar y comprarl...   
15               Acciones en GBM no se pueden vender.   
16  El mejor momento para vende

In [30]:
from pytrends.request import TrendReq

def get_google_trends_data(keywords, timeframe='today 1-m'):
    pytrends = TrendReq(hl='es-MX', tz=360)
    pytrends.build_payload(keywords, cat=0, timeframe=timeframe, geo='MX', gprop='')
    data = pytrends.interest_over_time()
    return data


keywords = ['BMV', 'inversiones', 'Acciones', 'comprar acciones']
trends_data = get_google_trends_data(keywords)

/opt/homebrew/anaconda3/lib/python3.12/site-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


In [32]:
# Verificar si se obtuvieron datos
if trends_data.empty:
    print("No se obtuvieron datos de Google Trends. Verifica las palabras clave y el timeframe.")
else:
    
    trends_data.reset_index(inplace=True)
    
    print("Datos originales:")
    print(trends_data.head())
    
    # Calcular el cambio porcentual para cada palabra clave
    for keyword in keywords:
        trends_data[f'{keyword}_pct_change'] = trends_data[keyword].pct_change() * 100
    
    # Definir cambios significativos
    significant_threshold = 51 
    
    # Crear columnas que indiquen si hubo un aumento significativo
    for keyword in keywords:
        trends_data[f'{keyword}_sig_increase'] = trends_data[f'{keyword}_pct_change'].apply(
            lambda x: 1 if x > significant_threshold else 0
        )
    # Crear una columna que sume los aumentos significativos de todas las palabras clave
    trend_sig_columns = [f'{keyword}_sig_increase' for keyword in keywords]
    trends_data['Google_Trends_Significant_Increase'] = trends_data[trend_sig_columns].sum(axis=1)
    
    trends_data['date'] = pd.to_datetime(trends_data['date']).dt.normalize()
    

Datos originales:
        date  BMV  inversiones  Acciones  comprar acciones  isPartial
0 2024-09-30    2            8        63                 0      False
1 2024-10-01    3            6        54                 1      False
2 2024-10-02    4            8        87                 1      False
3 2024-10-03    4            9        92                 0      False
4 2024-10-04    3           10        78                 1      False

Datos con cambios porcentuales y aumentos significativos:
        date  BMV  inversiones  Acciones  comprar acciones  isPartial  \
0 2024-09-30    2            8        63                 0      False   
1 2024-10-01    3            6        54                 1      False   
2 2024-10-02    4            8        87                 1      False   
3 2024-10-03    4            9        92                 0      False   
4 2024-10-04    3           10        78                 1      False   

   BMV_pct_change  inversiones_pct_change  Acciones_pct_change  

In [34]:
trends_data['date'] = pd.to_datetime(trends_data['date']).dt.normalize()
trends_data.rename(columns={'date': 'Date'}, inplace=True)

        Date  BMV  inversiones  Acciones  comprar acciones  isPartial  \
0 2024-09-30    2            8        63                 0      False   
1 2024-10-01    3            6        54                 1      False   
2 2024-10-02    4            8        87                 1      False   
3 2024-10-03    4            9        92                 0      False   
4 2024-10-04    3           10        78                 1      False   

   BMV_pct_change  inversiones_pct_change  Acciones_pct_change  \
0             NaN                     NaN                  NaN   
1       50.000000              -25.000000           -14.285714   
2       33.333333               33.333333            61.111111   
3        0.000000               12.500000             5.747126   
4      -25.000000               11.111111           -15.217391   

   comprar acciones_pct_change  BMV_sig_increase  inversiones_sig_increase  \
0                          NaN                 0                         0   
1       

In [36]:
trends_daily = trends_data[['Date', 'Google_Trends_Significant_Increase']]

         Date  Google_Trends_Significant_Increase
0  2024-09-30                                   0
1  2024-10-01                                   1
2  2024-10-02                                   1
3  2024-10-03                                   0
4  2024-10-04                                   1
5  2024-10-05                                   0
6  2024-10-06                                   0
7  2024-10-07                                   3
8  2024-10-08                                   0
9  2024-10-09                                   1
10 2024-10-10                                   0
11 2024-10-11                                   0
12 2024-10-12                                   0
13 2024-10-13                                   2
14 2024-10-14                                   1
15 2024-10-15                                   1
16 2024-10-16                                   0
17 2024-10-17                                   1
18 2024-10-18                                   0


In [38]:
# 4.1 Aggregate News Sentiment by Date
news_daily_sentiment = news_df.groupby('Date')['sentiment_score'].mean().reset_index()
print(news_daily_sentiment)

         Date  sentiment_score
0  2024-08-30        -1.000000
1  2024-09-10        -1.000000
2  2024-09-20        -1.000000
3  2024-10-09        -1.000000
4  2024-10-11        -1.000000
5  2024-10-16         0.000000
6  2024-10-18         0.000000
7  2024-10-19        -1.000000
8  2024-10-21        -0.333333
9  2024-10-22        -1.000000
10 2024-10-23        -0.800000
11 2024-10-24        -0.777778
12 2024-10-25        -0.600000
13 2024-10-26        -1.000000
14 2024-10-27        -1.000000
15 2024-10-28        -1.000000
16 2024-10-29        -0.636364
17 2024-10-30        -0.800000


In [52]:
# 4.2 Aggregate Reddit Sentiment by Date
reddit_daily_sentiment = reddit_df.groupby('Date')['sentiment_score'].mean().reset_index()
print(reddit_daily_sentiment)


         Date  sentiment_score
0  2022-06-03             -1.0
1  2023-06-10             -1.0
2  2023-06-17             -1.0
3  2023-08-21             -1.0
4  2024-01-31             -1.0
5  2024-02-02             -1.0
6  2024-02-11             -1.0
7  2024-02-17             -1.0
8  2024-03-25              1.0
9  2024-03-31             -1.0
10 2024-04-08             -1.0
11 2024-05-04              1.0
12 2024-06-11             -1.0
13 2024-06-27             -1.0
14 2024-08-15             -0.5
15 2024-10-01             -1.0
16 2024-10-03              1.0
17 2024-10-05             -1.0
18 2024-10-16             -1.0
19 2024-10-30             -1.0


In [54]:
#Volatility
ipc = yf.download('^MXX', start=start_date_str, end=end_date_str, interval='1d')
ipc.reset_index(inplace=True)
ipc['Date'] = pd.to_datetime(ipc['Date'])
ipc['Returns'] = ipc['Close'].pct_change()
ipc['Volatility'] = ipc['Returns'].rolling(window=2).std() * np.sqrt(2)

print(ipc['Volatility'])

default_volatility = 0.01
ipc['Volatility'].fillna(default_volatility, inplace=True)

print("ipc shape:", ipc.shape)
print(ipc[['Date', 'Volatility']].head())

[*********************100%***********************]  1 of 1 completed

0           NaN
1           NaN
2      0.021525
3      0.000850
4      0.000356
         ...   
490    0.004121
491    0.002320
492    0.006430
493    0.000834
494    0.013162
Name: Volatility, Length: 495, dtype: float64
ipc shape: (495, 9)
        Date  Volatility
0 2022-11-10    0.010000
1 2022-11-11    0.010000
2 2022-11-14    0.021525
3 2022-11-15    0.000850
4 2022-11-16    0.000356



/var/folders/6q/jns7j9_90psfzrg2_z5bct3h0000gn/T/ipykernel_28322/2117013522.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ipc['Volatility'].fillna(default_volatility, inplace=True)


In [56]:
# Merge News and Reddit sentiment
sentiment_data = pd.merge(
    news_daily_sentiment,
    reddit_daily_sentiment,
    on='Date',
    how='outer',
    suffixes=('_news', '_reddit')
)

sentiment_data = pd.merge(
    sentiment_data,
    trends_daily,
    on='Date',
    how='outer'
)


sentiment_data.fillna(method='ffill', inplace=True)
sentiment_data.dropna(inplace=True)

# Merge w md
ipc_sentiment = pd.merge(sentiment_data, ipc[['Date', 'Volatility']], on='Date', how='left')
ipc_sentiment.dropna(inplace=True)

/var/folders/6q/jns7j9_90psfzrg2_z5bct3h0000gn/T/ipykernel_28322/2750046490.py:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sentiment_data.fillna(method='ffill', inplace=True)


In [125]:

ipc['Returns'] = ipc['Close'].pct_change()


ipc['Volatility'] = ipc['Returns'].rolling(window=30).std() * np.sqrt(30)
ipc['Volatility'].fillna(ipc['Volatility'].mean(), inplace=True)

ipc['Volume'] = ipc['Volume'].astype(float)

ipc['Volume_30d_avg'] = ipc['Volume'].rolling(window=30).mean()
ipc['Volume_vs_30d_avg'] = ipc['Volume'] / ipc['Volume_30d_avg']


ipc['Momentum'] = ipc['Close'] - ipc['Close'].shift(1)
ipc['Momentum_30d_avg'] = ipc['Momentum'].rolling(window=30).mean()
ipc['Momentum_vs_30d_avg'] = ipc['Momentum'] / ipc['Momentum_30d_avg']


print("Columns in ipc DataFrame:")
print(ipc.columns.tolist())

# selecting ipc_metrics
ipc_metrics = ipc[['Date', 'Volatility', 'Volume', 'Close', 'Momentum', 'Volume_30d_avg',
                   'Volume_vs_30d_avg', 'Momentum_30d_avg',
                   'Momentum_vs_30d_avg',]]

# merging ipc_metrics with sentiment_data
ipc_sentiment = pd.merge(
    sentiment_data,
    ipc_metrics,
    on='Date',
    how='outer'
)

ipc_sentiment.sort_values('Date', inplace=True)
ipc_sentiment.fillna(method='ffill', inplace=True)
ipc_sentiment.fillna(method='bfill', inplace=True)

# Invert Volatility
ipc_sentiment['Volatility_inv'] = 1 / ipc_sentiment['Volatility']
ipc_sentiment.replace([np.inf, -np.inf], np.nan, inplace=True)
ipc_sentiment['Volatility_inv'].fillna(ipc_sentiment['Volatility_inv'].mean(), inplace=True)


Columns in ipc DataFrame:
['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Returns', 'Volatility', 'Volume_30d_avg', 'Volume_90d_avg', 'Volume_vs_30d_avg', 'Volume_vs_90d_avg', 'Momentum', 'Momentum_30d_avg', 'Momentum_90d_avg', 'Momentum_vs_30d_avg', 'Momentum_vs_90d_avg']


/var/folders/6q/jns7j9_90psfzrg2_z5bct3h0000gn/T/ipykernel_28322/512060625.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ipc['Volatility'].fillna(ipc['Volatility'].mean(), inplace=True)
/var/folders/6q/jns7j9_90psfzrg2_z5bct3h0000gn/T/ipykernel_28322/512060625.py:43: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ipc_sentiment.fillna(method='ffill', inplace=True)
/var/folders/6q/jns7j9_90psfzrg2_z5bct3h0000gn/T/ipykernel_28322/5120

In [127]:
features_to_normalize = [
    'sentiment_score_news',
    'sentiment_score_reddit',
    'Google_Trends_Significant_Increase',
    'Volatility',
    'Volume',
    'Momentum',
    'Volume_vs_30d_avg',
    'Momentum_vs_30d_avg',
    'Volatility_inv'
]

scaler = MinMaxScaler()

# Apply scaling
ipc_sentiment_scaled = ipc_sentiment.copy()
ipc_sentiment_scaled[features_to_normalize] = scaler.fit_transform(
    ipc_sentiment[features_to_normalize]
)

print(ipc_sentiment_scaled)

          Date  sentiment_score_news  sentiment_score_reddit  \
0   2022-11-10              0.000000                    0.25   
1   2022-11-11              0.000000                    0.25   
2   2022-11-14              0.000000                    0.25   
3   2022-11-15              0.000000                    0.25   
4   2022-11-16              0.000000                    0.25   
..         ...                   ...                     ...   
501 2024-10-27              0.000000                    0.00   
502 2024-10-28              0.000000                    0.00   
503 2024-10-29              0.363636                    0.00   
504 2024-10-30              0.200000                    0.00   
505 2024-10-31              0.200000                    0.00   

     Google_Trends_Significant_Increase  Volatility    Volume         Close  \
0                              0.000000    0.298831  0.073963  51031.710938   
1                              0.000000    0.298831  0.083060  51959.1992

In [129]:
weights = {
    'sentiment_score_news': 0.25,
    'sentiment_score_reddit': 0.25,
    'Google_Trends_Significant_Increase':0.1,
#25% Volatility
    'Volatility_inv': 0.25,
#25% market/volume
    'Volume_vs_30d_avg': 0.15,
    'Momentum_vs_30d_avg': 0.1,
}

features = list(weights.keys())

ipc_sentiment_scaled.set_index('Date', inplace=True)

def adjust_weights(row, weights):
    available_weights = {}
    total_weight = 0

    for feature, weight in weights.items():
        if pd.notna(row[feature]):
            available_weights[feature] = weight
            total_weight += weight

    # Normalize weights
    for feature in available_weights:
        available_weights[feature] /= total_weight

    return available_weights

def calculate_sentiment_index(row):
    adj_weights = adjust_weights(row, weights)
    sentiment_index = 0
    for feature, weight in adj_weights.items():
        sentiment_index += row[feature] * weight
    return sentiment_index * 100  # Scale to 0-100

ipc_sentiment_scaled['Sentiment_Index'] = ipc_sentiment_scaled.apply(calculate_sentiment_index, axis=1)

ipc_sentiment_scaled.reset_index(inplace=True)

In [135]:
# Create a new DataFrame with only 'Date' and 'Sentiment_Index' to save csv
sentiment_index_df = ipc_sentiment_scaled[['Date', 'Sentiment_Index']]

sentiment_index_df.to_csv('sentiment_index.csv', index=False)